In [1]:
import numpy as np
import pandas as pd
import re
import warnings
import datetime
from sklearn.ensemble import GradientBoostingRegressor
warnings.filterwarnings('ignore')

In [2]:
# Import the data
data = pd.read_csv('train_2011_2012_2013.csv', sep=';')

In [3]:
# We keep only relevant columns
data = data[['DATE', 'ASS_ASSIGNMENT', 'CSPL_RECEIVED_CALLS']]

In [4]:
# We group by DATE and ASS_ASSIGNMENT and sum over CSPL_RECEIVED_CALLS
groupbydata = data.groupby(['DATE', 'ASS_ASSIGNMENT'])['CSPL_RECEIVED_CALLS'].sum()
newarr = []
for key, value in groupbydata.items():
    newarr.append(list(key) + [value])
data = pd.DataFrame(newarr)
data.columns = ['DATE', 'ASS_ASSIGNMENT', 'CSPL_RECEIVED_CALLS']

In [5]:
# 'Evenements' and 'Gestion Amex' don't appear in submission.txt
data = data[~data.ASS_ASSIGNMENT.isin(['Evenements', 'Gestion Amex'])]

In [6]:
# Get the ASS_ASSIGNMENT
class_label = np.unique(data.ASS_ASSIGNMENT)

In [7]:
def simplifyDATE(date):
    """
    Parameters:
    -----------
    date, with format YYYY-MM-DD hh:mm:00.000
    
    Return    :
    -----------
    date, with format YYYYMMDDhhmm
    """
    return re.sub(r'[^\w]', '', date)[:12]

def splitDate(data):
    """
    Parameters:
    -----------
    dataframe
    
    Return    :
    -----------
    Add columns to data (YEAR, MONTH, DAY, HOUR, MIN)
    """
    data['YEAR']  = data['DATE_SIMPLIFY'].map(lambda x: int(x[:4]))
    data['MONTH'] = data['DATE_SIMPLIFY'].map(lambda x: int(x[4:6]))
    data['DAY']   = data['DATE_SIMPLIFY'].map(lambda x: int(x[6:8]))
    data['HOUR']  = data['DATE_SIMPLIFY'].map(lambda x: int(x[8:10]))
    data['MIN']   = data['DATE_SIMPLIFY'].map(lambda x: int(x[10:12]))

def isWeekEnd(x):
    """
    Parameters:
    -----------
    date, with format YYYYMMDDhhmm
    
    Return    :
    -----------
    1 if date corresponds to a weekend, 0 otherwise
    """
    return 1 if datetime.datetime(int(x[:4]), int(x[4:6]), int(x[6:8]), int(x[8:10]), int(x[10:12])).weekday() in [5, 6] else 0

def getWeekEnd(data):
    """
    Parameters:
    -----------
    dataframe
    
    Return    :
    -----------
    Add a column to dataframe (WEEKEND), with values 1 if the row correponds to a weekend, 0 otherwise
    """
    data['WEEKEND'] = data['DATE_SIMPLIFY'].map(isWeekEnd)
    
def getYearMonthDay(data):
    """
    Parameters:
    -----------
    date, with format YYYYMMDDhhmm
    
    Return    :
    -----------
    date, with format YYYYMMDD
    """
    data['YEAR_MONTH_DAY'] = data['DATE_SIMPLIFY'].map(lambda x: int(x[:8]))
    
def dummifyWeekday(data):
    """
    Parameters:
    -----------
    dataframe
    
    Return    :
    -----------
    Add columns to dataframe (WEEKDAYi), with values 1 if the row correponds to WEEKDAYi (WEEKDAY0 for Monday), 
    (WEEKDAY1 for Tuesday)...
    0 otherwise
    """
    dummies = pd.get_dummies(data['WEEKDAY'])
    dummies.columns = ['WEEKDAY' + str(c) for c in dummies.columns]
    data[dummies.columns] = dummies
    
def ferie(x):
    """
    Parameters:
    -----------
    date, with format YYYYMMDD
    
    Return    :
    -----------
    1 if date is a day off, 0 otherwise
    """
    DAY_OFF = \
    ['20110101', '20110424', '20110425', '20110501', '20110508', '20110602', 
     '20110612', '20110613', '20110714', '20110815', '20111101', '20111111', 
     '20111225', '20120101', '20120408', '20120409', '20120501', '20120508', 
     '20120517', '20120527', '20120528', '20120714', '20120815', '20121101', 
     '20121111', '20121225', '20130101', '20130331', '20130401', '20130501', 
     '20130508', '20130509', '20130519', '20130520', '20130714', '20130815', 
     '20131101', '20131111', '20131225']
    return 1 if x in DAY_OFF else 0

def addZero(x):
    """
    Parameters:
    -----------
    An integer [0-99]
    
    Return    :
    -----------
    A string, with a 0 in first index if integer < 10
    """
    return '0' + str(x) if x < 10 else str(x)

mappingHourMin = {addZero(hour) + mn: 2*index_hour + index_mn for index_hour, hour in enumerate(range(24)) for index_mn, mn in enumerate(['00', '30'])}

def getMappingHourmin(x):
    """
    Parameters:
    -----------
    Time with format hhmm
    
    Return    :
    -----------
    An integer, 0 for 0000, 1 for 0030, 2 for 0100, ..., 47 for 2330
    """
    return mappingHourMin[x]

def getNumberDaysSinceBeg(date):
    """
    Parameters:
    -----------
    date, with format YYYYMMDD
    
    Return    :
    -----------
    Number of day since January, 1st 2011
    """
    return (datetime.date(int(str(date)[:4]), int(str(date)[4:6]), int(str(date)[6:8])) - datetime.date(2011, 1, 1)).days

def isAfterHoliday(date):
    """
    Return 1 if date is a day after a day off, 0 otherwise
    """
    if ferie(str(int(date-1))) == 1 and getWeekDay(str(int(date)))!= 5 and getWeekDay(str(int(date)))!= 6 and ferie(str(int(date))) == 0:
        return 1
    return 0

def getWeekDay(date):
    """
    Parameters:
    -----------
    date, with format YYYYMMDD
    
    Return    :
    -----------
    Integer, 0 for Monday, 1 for Tuesday, ..., 6 for Sunday
    """

    return datetime.datetime(int(str(date)[:4]), int(str(date)[4:6]), int(str(date)[6:8])).weekday()
    
def getAllDates(withHourMin=True, start_date='2011-01-01', end_date='2013-12-31'):
    """
    Yields all dates between start_date and end_date as a String
    """
    if withHourMin:
        for ymd in pd.date_range(start=start_date, end=end_date, freq='D'):
            for h in [addZero(i) for i in range(24)]:
                for m in ['00', '30']:
                    yield str(ymd.year) + addZero(ymd.month) + addZero(ymd.day) + h + m
    else:
        for ymd in pd.date_range(start=start_date, end=end_date, freq='D'):
            yield str(ymd.year) + addZero(ymd.month) + addZero(ymd.day)

def fromDATE_SIMPLIFYtoDATE(ds):
    """
    Parameters:
    -----------
    date, with format YYYYMMDDhhmm
    
    Return    :
    -----------
    date, with format YYYY-MM-DD hh:mm:00.000
    """
    s = str(ds)
    return s[:4]+'-'+s[4:6]+'-'+s[6:8]+' '+s[8:10]+':'+s[10:12]+':00.000'

def score(y_true_list, y_pred_list):
    """
    Return the score as defined in the challenge description
    """
    alpha = 0.1
    return np.mean([np.exp(alpha*(y_true - y_pred)) - alpha*(y_true - y_pred) - 1 for (y_true, y_pred) in zip(y_true_list, y_pred_list)])

def getX_Y(data, columns_X):
    """
    Parameters:
    -----------
    dataframe,
    relevant columns as an array
    
    Return    :
    -----------
    X, y
    """
    X, y = data[columns_X].values, data['CSPL_RECEIVED_CALLS'].values
    return X, y

def getX(data, columns_X):
    """
    Parameters:
    -----------
    dataframe,
    relevant columns as an array
    
    Return    :
    -----------
    X
    """
    X = data[columns_X].values
    return X

def save(data, title):
    """
    Helper function to save submission.txt, once the predictions done
    """
    data.to_csv(title + '.txt', sep='\t', index=False)
    
def tranformData(data):
    """
    Parameters:
    -----------
    dataframe
    
    Return    :
    -----------
    dataframe with a set of transformations (add several columns)
    """
    if not 'DATE_SIMPLIFY' in data.columns:
        data['DATE_SIMPLIFY'] = data['DATE'].map(simplifyDATE)
    data['DATE_SIMPLIFY'] = data['DATE_SIMPLIFY'].map(lambda x: str(x))
    data['HOURMIN'] = data['DATE_SIMPLIFY'].map(lambda x: x[8:])
    data['WEEKDAY'] = data['DATE_SIMPLIFY'].map(getWeekDay)
    data['HOLIDAY'] = data['DATE_SIMPLIFY'].map(ferie)
    splitDate(data)
    getWeekEnd(data)
    getYearMonthDay(data)
    data['IS_AFTER_HOLIDAY'] = data['YEAR_MONTH_DAY'].map(isAfterHoliday)
    data['HOURMIN_HL'] = data['HOURMIN'].map(getMappingHourmin)
    data['DAYS_SINCE_BEG'] = data['YEAR_MONTH_DAY'].map(getNumberDaysSinceBeg)
    dummifyWeekday(data)

In [8]:
# Import submission.txt
sub = pd.read_csv('submission.txt', sep='\t')

In [9]:
# We apply the same transformation we applied for data
tranformData(sub)

In [10]:
# List of all weeks of the submission file
weeks_sub = np.array([list(np.unique(sub.YEAR_MONTH_DAY)[7*i:7*(i+1)]) for i in range(12)])

In [11]:
def getDataForAllDates(data, class_lab, end_date='2013-12-28'):
    """
    Cette fonction sert à retourner pour un ASS_ASSGNEMENT un dataframe à deux colonnes, la première colonne est
    DATE_SIMPLIFY, avec des dates du type 201212081330 (donc 8 décembre 2012 à 13h30) et pour deuxième colonne
    CSPL_RECEIVED_CALLS, i.e. le nombre d'appels reçu pour cette date et pour ce ASS_ASSGNEMENT. Si aucune donnée
    n'est fournie, on estime que le nombre d'appel reçu est zéro (cf les courbes précédentes pour la justification)
    """
    data['DATE_SIMPLIFY'] = data['DATE'].map(simplifyDATE)
    data_label = data.loc[data.ASS_ASSIGNMENT == class_lab]
    data_label_group_by = data_label.groupby(['DATE_SIMPLIFY'])['CSPL_RECEIVED_CALLS'].sum()

    dates_ = []
    values_ = []
    date_in_data_label = set([str(x) for x in data_label_group_by.index])
    for date in getAllDates(withHourMin=True, end_date=end_date):
        dates_.append(int(date))
        if date in date_in_data_label:
            values_.append(data_label_group_by[date])
        else:
            values_.append(0)

    data = pd.DataFrame(np.stack((dates_,values_), axis=1), columns=['DATE_SIMPLIFY', 'CSPL_RECEIVED_CALLS'])
    return data

In [12]:
def getPredictor(data, class_label, weeks_sub, verbose=True):
    """
    This function is our predictor. It trains a regressor per ASS_ASSIGNEMENT, in our case, we have chosen 
    Gradient Boosting Regresssor.In order to train this regressor, we need the list of all weeks for which 
    we have to predict the number of calls. This list is obtained through weeks_sub.
    Then, for each week_i that needs to be predicted, we use all the data available before week_i (included the 
    predictions we have already done) and do the training over it and eventually predict the number of calls. 
    We operate this wey until the last week to predict.
    To get an idea of our score, we put verbose = True, hence if the predictor is trained to predict the number 
    of calls for the week S (indeed, we don't have the true number of calls), then the predictor is tested on the 
    following week S+1 (for which we already have the true number of calls).
    """
    pred = {class_lab: {} for class_lab in class_label}
    relevant_columns = ['WEEKEND', 'HOLIDAY', 'IS_AFTER_HOLIDAY', 'HOURMIN_HL', 'DAYS_SINCE_BEG', 'WEEKDAY0', 'WEEKDAY1', 'WEEKDAY2',
       'WEEKDAY3', 'WEEKDAY4', 'WEEKDAY5', 'WEEKDAY6', 'MONTH']


    for class_lab in class_label:
        if verbose:
            print('We train our regressor for %s' %class_lab)
        
        # Pick only data for the class_lab and add the relevant columns
        data_ = getDataForAllDates(data, class_lab)
        tranformData(data_)
        
        for week in list(weeks_sub):
            # Train set data
            X_train, y_train = getX_Y(data_[data_.YEAR_MONTH_DAY < week[0]], columns_X=relevant_columns)
            
            # data with missing values that we have to predict 
            X_test = getX(data_[data_.YEAR_MONTH_DAY.isin(week)], columns_X=relevant_columns)
            
            # Train our regressor
            regressor = GradientBoostingRegressor(n_estimators=1000)
            regressor.fit(X_train, y_train)
            
            # Update the data with the prediction on the week
            data_.ix[data_.YEAR_MONTH_DAY.isin(week), 'CSPL_RECEIVED_CALLS'] = np.clip(regressor.predict(X_test), 0, None)
        
        
        data_['DATE'] = data_.DATE_SIMPLIFY.map(fromDATE_SIMPLIFYtoDATE)
                    
        for row in data_.iterrows():
            # Update the predictions
            pred[class_lab][row[1].DATE] = row[1].CSPL_RECEIVED_CALLS if row[1].CSPL_RECEIVED_CALLS >= 0 else 0
    
    return pred

In [13]:
# Train our regressors and fill submission values
predictors = getPredictor(data, class_label, weeks_sub, verbose=True)

We train our regressor for CAT
We train our regressor for CMS
We train our regressor for Crises
We train our regressor for Domicile
We train our regressor for Gestion
We train our regressor for Gestion - Accueil Telephonique
We train our regressor for Gestion Assurances
We train our regressor for Gestion Clients
We train our regressor for Gestion DZ
We train our regressor for Gestion Relation Clienteles
We train our regressor for Gestion Renault
We train our regressor for Japon
We train our regressor for Manager
We train our regressor for Mécanicien
We train our regressor for Médical
We train our regressor for Nuit
We train our regressor for Prestataires
We train our regressor for RENAULT
We train our regressor for RTC
We train our regressor for Regulation Medicale
We train our regressor for SAP
We train our regressor for Services
We train our regressor for Tech. Axa
We train our regressor for Tech. Inter
We train our regressor for Tech. Total
We train our regressor for Téléphonie


In [14]:
# Make our final predictions
sub = pd.read_csv('submission.txt', sep='\t')
coef_overestimation = 1.25
sub.prediction = sub.apply(lambda x: coef_overestimation*predictors[x['ASS_ASSIGNMENT']][x['DATE']], axis=1)
save(sub, "predictions")